In [ ]:
!{'python3 -m pip install dataflows==0.0.58'}
!{'python3 -m pip install -e git+https://github.com/OriHoch/dataflows-xlsx.git@master#egg=dataflows-xlsx'}

In [1]:
import getpass
CKAN_API_KEY = getpass.getpass("CKAN_API_KEY")

CKAN_API_KEY ····································


In [2]:
CKAN_URL='https://odata.org.il'

In [4]:
import requests
from dataflows import Flow, printer, checkpoint
from dataflows_xlsx import dump_to_path

def get_group_names():
    for group_name in requests.get(CKAN_URL+'/api/3/action/group_list', headers={'Authorization': CKAN_API_KEY}).json()['result']:
        yield {'name': group_name}
        
Flow(
    get_group_names(),
    checkpoint('group_names'),
    printer(tablefmt='html', num_rows=1)
).process()

saving checkpoint to: .checkpoints/group_names


#,name (string)
1,foi-201908-138
2,foi-201908-517
...,
2023,foi-201908-756


checkpoint saved: group_names


(<datapackage.package.Package at 0x7efdb2190438>, {})

In [8]:
import requests
from dataflows import Flow, printer, checkpoint
from dataflows_xlsx import dump_to_path

def get_group_details():
    for group in Flow(checkpoint('group_names')).results()[0][0]:
        group = requests.get(CKAN_URL+'/api/3/action/group_show?id={}&include_extras=true'.format(group['name']), headers={'Authorization': CKAN_API_KEY}).json()['result']
        group = {k: group[k] for k in group if k in [
            'display_name', 'package_count', 'name', 'state', 'extras', 'title', 'id', 'approval_status'
        ]}
        yield group

Flow(
    get_group_details(),
    checkpoint('all_group_details'),
    printer(tablefmt='html', num_rows=1)
).process()

saving checkpoint to: .checkpoints/all_group_details
using checkpoint data from .checkpoints/group_names


#,display_name (string),package_count (integer),name (string),state (string),extras (array),title (string),id (string),approval_status (string)
1,כפר הנוער מאיר שפיה,0,foi-201908-138,active,"[{'value': '04-6397760', 'state': 'active', 'key': 'Fax', 'revision_id': '0764667f-7e57-41c7-8c8a-5c ...",כפר הנוער מאיר שפיה,4979dd14-ab38-44b7-980b-394442814482,approved
2,פרזות חברה ממשלתית עירונית לשיכון ירושלים בעמ,0,foi-201908-517,active,"[{'value': None, 'state': 'active', 'key': 'Fax', 'revision_id': 'a8b6de87-14f7-40ab-a9af-b692afd40b ...",פרזות חברה ממשלתית עירונית לשיכון ירושלים בעמ,f52905cc-f97f-40d6-8cb6-c8e95f61f53b,approved
...,,,,,,,,
2023,תשתית - חברה לשירותי תחזוקה באשדוד בעמ,0,foi-201908-756,active,"[{'value': '08-8545026 ', 'state': 'active', 'key': 'Fax', 'revision_id': 'b5fe49ee-2de1-49d0-aeed-4 ...",תשתית - חברה לשירותי תחזוקה באשדוד בעמ,a9bc889a-2822-452b-9f61-0f9a064caf4b,approved


checkpoint saved: all_group_details


(<datapackage.package.Package at 0x7efdb1d8a6a0>, {})

In [19]:
from dataflows import Flow, checkpoint, printer, update_resource
from dataflows_xlsx import dump_to_path

all_extra_keys = set()

for group in Flow(checkpoint('all_group_details')).results()[0][0]:
    for extra in group['extras']:
        all_extra_keys.add(extra['key'])

all_capitalized_keys = set()
        
for key in all_extra_keys:
    if key in ['group_id', 'merged_with_json', 'match_type', 'entity_id', 'munged_title', 'language', '__type']: continue
    has_capitalized_key = False
    for tmpkey in all_extra_keys:
        if key == tmpkey: continue
        if tmpkey.lower() == tmpkey: continue
        if tmpkey.lower() == key.lower():
            has_capitalized_key = True
    if not has_capitalized_key:
        all_capitalized_keys.add(key)

def get_group_extras():
    for group in Flow(checkpoint('all_group_details')).results()[0][0]:
        group_extras = {extra['key'].lower(): extra['value'] for extra in group['extras']}
        for ckey in all_capitalized_keys:
            group[ckey] = group_extras.get(ckey.lower(), '')
        del group['extras']
        yield group

Flow(
    get_group_extras(),
    update_resource('res_1', name='all_groups', path='all_groups.xlsx'),
    dump_to_path('data/all_groups', format='xlsx'),
    printer(tablefmt='html', num_rows=1),
).process()

using checkpoint data from .checkpoints/all_group_details
using checkpoint data from .checkpoints/all_group_details


#,approval_status (string),display_name (string),id (string),name (string),package_count (integer),state (string),title (string),nid (string),Published (string),override_email (string),PaymentTreasury (string),OfficeTypeCode (string),websiteURL (string),email (string),override_foi_link (string),payments (string),form (string),PaymentPostalBank (string),FormFax (string),officenamecode (string),MMDSubjects (string),OfficeIcon (string),FormOnlineURL (string),GovXDescription (string),MMDTypes (string),forWizard (string),PaymentCash (string),ReceptionAddressNotes (string),FormEmail (string),PaymentBankTransfer (string),PaymentPhone (string),GovXContentSection (string),PaymentCheck (string),mmdOfficesTypes_tid (string),FormLetter (string),ReceptionPhoneNumbers (string),ManagerName (string),PaymentOnlineURL (string),Fax (string)
1,approved,כפר הנוער מאיר שפיה,4979dd14-ab38-44b7-980b-394442814482,foi-201908-138,0,active,כפר הנוער מאיר שפיה,None,Yes,None,None,6,ww.shfeyah.org.il,pazit@shfeyah.org.il,None,None,None,בנק המזרחי סניף 420 חדרה חשבון 572360,04-6397760,6005,None,None,~,None,None,No,None,כפר הנוער מאיר שפיה ד.נ. חוף הכרמל מיקוד 3080600,pazit@shfeyah.org.il,בנק המזרחי סניף 420 חדרה חשבון 572360,None,כפר הנוער מאיר שפיה הינו מוסד חינוכי ייחודי המתמחה במדעי החיים והחקלאות. בכפר בית ספר מקיף שש שנת ...,None,47,כפר הנוער מאיר שפיה ד.נ. חוף הכרמל מיקוד 3080600,04-6390750,פזית כהן,None,04-6397760
2,approved,פרזות חברה ממשלתית עירונית לשיכון ירושלים בעמ,f52905cc-f97f-40d6-8cb6-c8e95f61f53b,foi-201908-517,0,active,פרזות חברה ממשלתית עירונית לשיכון ירושלים בעמ,None,Yes,None,None,6,http://www.prazot.co.il,limorc@prazot.co.il,None,None,None,None,None,6006,None,None,None,None,None,No,None,"רח' ירמיהו 11, ירושלים 9446136",limorc@prazot.co.il,None,None,"פרזות חברה ממשלתית עירונית לשיכון ירושלים בע""מ היא זרוע לביצוע מדיניות משרד הבינוי והשיכון, המטפל ...",None,47,None,02-5016204,לימור כהן,None,None
...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023,approved,תשתית - חברה לשירותי תחזוקה באשדוד בעמ,a9bc889a-2822-452b-9f61-0f9a064caf4b,foi-201908-756,0,active,תשתית - חברה לשירותי תחזוקה באשדוד בעמ,None,Yes,None,None,10,None,davidd@ashdod.muni.il,None,None,None,None,08-8545026,10012,None,None,~,None,None,No,None,"רח' הגדוד העברי 10, ת.ד. 28, אשדוד 7710001",davidd@ashdod.muni.il,None,None,חברות עירוניות הן תאגידים עירוניים המוקמים על ידי רשויות מקומיות ומנוהלים על ידי אנשי מ ...,None,325,"רח' הגדוד העברי 10, ת.ד. 28, אשדוד 7710001",08-8545498/90,דוד דבש,None,08-8545026


(<datapackage.package.Package at 0x7efdb05be710>,
 {'count_of_rows': 2023,
  'bytes': 5154,
  'hash': 'f31efed747bda5a85ae865e81f26d7e8',
  'dataset_name': None})